In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/Users/macbookpro/Documents/sclable_ml_pipelines/scalable_ml_pipelines'

In [3]:
# Importing necessary modules and functions from the pipeline
from us_used_cars_ml_pipeline.components.data_cleaning import CleanData
from us_used_cars_ml_pipeline.utils.common import get_spark_session
from us_used_cars_ml_pipeline.config.configuration import ConfigurationManager


In [4]:
# Initializing the ConfigurationManager and fetching data ingestion configuration
config_manager = ConfigurationManager()
data_cleaning_config = config_manager.get_clean_data_config()

# Initializing the DataIngestion component with the configuration obtained
data_cleaning = CleanData(data_cleaning_config)

# Starting a Spark session for data processing
spark = get_spark_session()

# Reading the data from HDFS using the DataIngestion component
df = data_cleaning.read_data_from_hdfs(spark)

# Display the initial DataFrame to understand its structure and the data it contains
df.show(truncate=False)


[2023-10-08 17:01:56,415: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: config/config.yaml loaded successfully]
[2023-10-08 17:01:56,427: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: params.yaml loaded successfully]
[2023-10-08 17:01:56,428: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: schema.yaml loaded successfully]


23/10/08 17:01:58 WARN Utils: Your hostname, Macbooks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
23/10/08 17:01:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/08 17:01:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/08 17:02:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------+------------+----+----------+----------+---------------+-----+--------+-----------------+--------------------+------------+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
# Import necessary functions and types from PySpark for data cleaning
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, FloatType, BooleanType, DateType, DoubleType, StringType


In [6]:
# Utility functions for data cleaning and transformation

# Function to extract float numbers from strings
def to_float(col):
    return F.regexp_extract(col, r"(\d+\.?\d*)", 1).cast(FloatType())

# Function to extract integer numbers from strings
def to_int(col):
    return F.regexp_extract(col, r"(\d+)", 1).cast(IntegerType())

# Function to convert a column to boolean type
def to_bool(col):
    return col.cast(BooleanType())

# Function to split power and torque columns into value and RPM
def split_power_torque(df, col_name):
    value = F.regexp_extract(df[col_name], r"(\d+)", 1).cast(IntegerType())
    rpm = F.regexp_replace(F.regexp_extract(df[col_name], r"@ ([\d,]+)", 1), ",", "").cast(IntegerType())
    return df.withColumn(f"{col_name}_value", value).withColumn(f"{col_name}_rpm", rpm)


In [7]:
# Dictionary mapping column names to their conversion functions
conversion_dict = {
    'back_legroom': to_float,
    'bed_height': to_float,
    'bed_length': to_float,
    'front_legroom': to_float,
    'height': to_float,
    'length': to_float,
    'wheelbase': to_float,
    'width': to_float,
    'city_fuel_economy': to_float,
    'combine_fuel_economy': to_float,
    'daysonmarket': to_int,
    'engine_displacement': to_float,
    'fuel_tank_volume': to_float,
    'highway_fuel_economy': to_float,
    'horsepower': to_int,
    'latitude': to_float,
    'longitude': to_float,
    'mileage': to_float,
    'owner_count': to_int,
    'price': to_float,
    'savings_amount': to_float,
    'seller_rating': to_float,
    'year': to_int,
    'fleet': to_bool,
    'frame_damaged': to_bool,
    'franchise_dealer': to_bool,
    'has_accidents': to_bool,
    'isCab': to_bool,
    'is_certified': to_bool,
    'is_cpo': to_bool,
    'is_new': to_bool,
    'is_oemcpo': to_bool,
    'salvage': to_bool,
    'theft_title': to_bool
}

# Apply conversion functions to corresponding columns
for col, func in conversion_dict.items():
    df = df.withColumn(col, func(df[col]))

# Convert listed_date to DateType
df = df.withColumn('listed_date', F.to_date(df['listed_date'], 'yyyy-MM-dd'))

# Split power and torque into value and rpm, and add new columns
df = split_power_torque(df, 'power')
df = split_power_torque(df, 'torque')

# Cleaning maximum_seating and converting to Integer
df = df.withColumn('maximum_seating', F.regexp_replace(F.col('maximum_seating'), '[^\d]+', '').cast(IntegerType()))

In [8]:
# Displaying the cleaned DataFrame
df.show()

+-----------------+------------+----+----------+----------+---------------+-----+--------+-----------------+--------------------+------------+----------+--------------------+----------------+-------------------+-----------+--------------------+-----+-------------+----------------+--------------+-------------+----------------+---------+-------------+------+--------------------+----------+--------------------+-----+------------+------+------+---------+--------+------+-----------+-------------+----------+---------+--------------------+--------------------+----------+---------------+-------+------------------+-----------+------------------+-------+-------+--------------+-------------+------+-------------------+-----------+--------------------+------------+--------------------+------+--------------------+-----------------------+------------+--------------------+---------+-----+----+-----------+---------+------------+----------+
|              vin|back_legroom| bed|bed_height|bed_length|    

In [9]:
# Selecting and describing only numeric columns for initial analysis
numeric_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, (DoubleType, IntegerType, FloatType))]
df.select(numeric_cols).describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+------------------+-------------------+------------------+-----------------+-----------------+--------------------+-----------------+------------------+------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-----------------+------------------+------------------+--------------------+------------------+--------------------+------------------+
|summary|      back_legroom|        bed_height|        bed_length| city_fuel_economy|combine_fuel_economy|      daysonmarket|engine_displacement|     front_legroom| fuel_tank_volume|           height|highway_fuel_economy|       horsepower|          latitude|            length|           longitude|    maximum_seating|             mileage|         owner_count|               price|     savings_amount|       seller_rating|        wheelbase| 

In [10]:
# Selecting and describing only string columns for initial analysis
string_cols = [field.name for field in df.schema.fields if isinstance(field.dataType, StringType)]
df.select(string_cols).describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----------------------+--------------------+--------------------+
|summary|                 vin|                 bed|           body_type|               cabin|                city|        dealer_zip|         description|    engine_cylinders|         engine_type|      exterior_color|franchise_make|           fuel_type|      interior_color|       listing_color|          listing_id|    main_picture_url|       major_options|        

In [ ]:
# Reading the cleaned Parquet data back from HDFS for further analysis or use
cleaned_data_path = 'hdfs://localhost:9000/path'  # specify the exact path where you have stored the Parquet file
cleaned_df = spark.read.parquet(cleaned_data_path)

# Show the DataFrame to verify it has been loaded correctly
cleaned_df.show()

# You can now proceed with further analysis or processing on the cleaned_df DataFrame
